# Assignment 1.2: Word2vec preprocessing (20 points)

Preprocessing is not the most exciting part of NLP, but it is still one of the most important ones. Your task is to preprocess raw text (you can use your own, or [this one](http://mattmahoney.net/dc/text8.zip). For this task text preprocessing mostly consists of:

1. cleaning (mostly, if your dataset is from social media or parsed from the internet)
1. tokenization
1. building the vocabulary and choosing its size. Use only high-frequency words, change all other words to UNK or handle it in your own manner. You can use `collections.Counter` for that.
1. assigning each token a number (numericalization). In other words, make word2index и index2word objects.
1. data structuring and batching - make X and y matrices generator for word2vec (explained in more details below)

**ATTN!:** If you use your own data, please, attach a download link. 

Your goal is to make **Batcher** class which returns two numpy tensors with word indices. It should be possible to use one for word2vec training. You can implement batcher for Skip-Gram or CBOW architecture, the picture below can be helpful to remember the difference.

![text](https://raw.githubusercontent.com/deepmipt/deep-nlp-seminars/651804899d05b96fc72b9474404fab330365ca09/seminar_02/pics/architecture.png)

There are several ways to do it right. Shapes could be `x_batch.shape = (batch_size, 2*window_size)`, `y_batch.shape = (batch_size,)` for CBOW or `(batch_size,)`, `(batch_size, 2*window_size)` for Skip-Gram. You should **not** do negative sampling here.

They should be adequately parametrized: CBOW(window_size, ...), SkipGram(window_size, ...). You should implement only one batcher in this task; and it's up to you which one to chose.

Useful links:
1. [Word2Vec Tutorial - The Skip-Gram Model](http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/)
1. [Efficient Estimation of Word Representations in Vector Space](https://arxiv.org/pdf/1301.3781.pdf)
1. [Distributed Representations of Words and Phrases and their Compositionality](http://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf)

You can write the code in this notebook, or in a separate file. It can be reused for the next task. The result of your work should represent that your batch has a proper structure (right shapes) and content (words should be from one context, not some random indices). To show that, translate indices back to words and print them to show something like this:

```
text = ['first', 'used', 'against', 'early', 'working', 'class', 'radicals', 'including']

window_size = 2

# CBOW:
indices_to_words(x_batch) = \
        [['first', 'used', 'early', 'working'],
        ['used', 'against', 'working', 'class'],
        ['against', 'early', 'class', 'radicals'],
        ['early', 'working', 'radicals', 'including']]

indices_to_words(labels_batch) = ['against', 'early', 'working', 'class']
```

In [1]:
import numpy as np
from pprint import pprint
from collections import Counter

In [2]:
class CBOWBatcher:
    def __init__(
        self, corpus, window_size, batch_size,
        vocabulary_size=20000
    ):
        self.corpus = corpus
        self.window_size = window_size
        self.batch_size = batch_size
        self.unk_token = '<UNK>'
        self.word2index = {
            word: index for index, (word, _)
            in enumerate(sorted(Counter(
                self.corpus
            ).items(), key=lambda x: x[1], reverse=True)[:vocabulary_size-1])
        }
        self.word2index[self.unk_token] = len(self.word2index)
        self.index2word = {
            index: word for word, index in self.word2index.items()
        }
        
    def __iter__(self):
        # permutatuion to shuffle our data
        self.permutation = np.random.permutation(
            np.arange(self.window_size, len(self.corpus) - self.window_size)
        )
        self.batch_number = 0
        return self
    
    def __next__(self):
        if self.batch_number < len(self.permutation) / self.batch_size:
            X = np.array(
                [
                    [
                        self.word2index.get(self.corpus[j], self.word2index[self.unk_token])
                        for j in range(i - self.window_size, i + self.window_size + 1)
                        if j != i
                    ] for i in self.permutation[self.batch_number * self.batch_size:
                                                (self.batch_number + 1) * self.batch_size]
                ]
            )
        
            y = np.array(
                [
                    self.word2index.get(self.corpus[i], self.word2index[self.unk_token])
                    for i in self.permutation[self.batch_number * self.batch_size:
                                                (self.batch_number + 1) * self.batch_size]
                ]
            )
            self.batch_number += 1
            return X, y
        else:
            raise StopIteration

In [3]:
text = ['first', 'used', 'against', 'early', 'working', 'class', 'radicals', 'including']
window_size = 2
batch_size = 4

batcher = CBOWBatcher(text, window_size, batch_size)

In [4]:
def print_batch(X_batch, y_batch):
    print('X_batch = ')
    print(X_batch)
    pprint([[batcher.index2word[x] for x in X] for X in X_batch])
    print()
    print('y_batch = ', y_batch)
    print([batcher.index2word[y] for y in y_batch])
    print()

for X_batch, y_batch in batcher:
    print_batch(X_batch, y_batch)

X_batch = 
[[0 1 3 4]
 [1 2 4 5]
 [2 3 5 6]
 [3 4 6 7]]
[['first', 'used', 'early', 'working'],
 ['used', 'against', 'working', 'class'],
 ['against', 'early', 'class', 'radicals'],
 ['early', 'working', 'radicals', 'including']]

y_batch =  [2 3 4 5]
['against', 'early', 'working', 'class']



In [5]:
with open('text8') as inf:
    corpus = inf.read().split()

In [6]:
window_size = 2
batch_size = 5

batcher = CBOWBatcher(corpus, window_size, batch_size, vocabulary_size=10000)

for i, (X_batch, y_batch) in enumerate(batcher):
    print_batch(X_batch, y_batch)
    
    if i == 1:
        break

X_batch = 
[[  296    18 10000     2]
 [  765     0 10000  2057]
 [ 1314  3802     5   283]
 [10000  1413 10000     6]
 [ 1844   216  1371     0]]
[['written', 'by', '<UNK>', 'and'],
 ['speed', 'the', '<UNK>', 'cars'],
 ['net', 'servers', 'a', 'player'],
 ['<UNK>', 'armed', '<UNK>', 'to'],
 ['swedish', 'military', 'declared', 'the']]

y_batch =  [859  50  61 368  46]
['richard', 'more', 'when', 'forces', 'they']

X_batch = 
[[   26    73    88     5]
 [ 1304     2   132   403]
 [    1  5822 10000  5822]
 [ 1996    26 10000    34]
 [    0  5241    31     6]]
[['it', 'would', 'called', 'a'],
 ['standards', 'and', 'same', 'software'],
 ['of', 'paint', '<UNK>', 'paint'],
 ['weeks', 'it', '<UNK>', 'at'],
 ['the', 'default', 'be', 'to']]

y_batch =  [ 31   0 364 620 279]
['be', 'the', 'especially', 'eventually', 'should']

